# SELENIUM EXTRACT URL

In [2]:
!pip install gdown
!pip install pandas
!pip install requests
!pip install pyyaml
!pip install selenium
!pip install stem

/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/.venv/bin/pip: line 2: /Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/7_Project_Cdiscount/.venv/bin/python3.11: No such file or directory
/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/.venv/bin/pip: line 2: exec: /Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/7_Project_Cdiscount/.venv/bin/python3.11: cannot execute: No such file or directory
/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/.venv/bin/pip: line 2: /Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/7_Project_Cdiscount/.venv/bin/python3.11: No such file or directory
/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/.venv/b

In [3]:
import gdown
import pandas as pd
import requests
import random
import time
import yaml
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from stem import Signal
from stem.control import Controller

ModuleNotFoundError: No module named 'gdown'

In [3]:
url = "https://drive.google.com/uc?id=1ZTdXDrX_p9t_zLqlpxDh3JT9L8FoevOx&export=download"
output = "product_ids.csv"
gdown.download(url, output, quiet=False)

df_pdt_id = pd.read_csv(output, sep=';')
df_pdt_id.head()

Downloading...
From: https://drive.google.com/uc?id=1ZTdXDrX_p9t_zLqlpxDh3JT9L8FoevOx&export=download
To: /Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/product_ids.csv
100%|██████████████████████████████████████| 80.7k/80.7k [00:00<00:00, 31.0MB/s]


,product_id
0,003411COBURG
1,016642F
2,0191ENFI
3,0355E4PT
4,0397ENSE


In [95]:
# Obtenir les 500 premiers IDs de produit

#item_list = df_pdt_id['product_id'].head(5).tolist()
#item_list = df_pdt_id['product_id'].sample(n=5, random_state=1).tolist()

#item_list = df_pdt_id['product_id'].tolist()
#item_list_1 = df_pdt_id['product_id'].tolist()[178:500]
#item_list_2 = df_pdt_id['product_id'].tolist()[501:1000]
#item_list_3 = df_pdt_id['product_id'].tolist()[1001:1500]
#item_list_4 = df_pdt_id['product_id'].tolist()[1501:2000]
#item_list_5 = df_pdt_id['product_id'].tolist()[2001:2500]
#item_list_6 = df_pdt_id['product_id'].tolist()[2501:3000]
#item_list_7 = df_pdt_id['product_id'].tolist()[3001:3500]
#item_list_8 = df_pdt_id['product_id'].tolist()[3501:4000]
#item_list_9 = df_pdt_id['product_id'].tolist()[4001:4500]
item_list_10 = df_pdt_id['product_id'].tolist()[4501:4962]

item_list_10

['UNI3665884035592',
 'UNI3665884049179',
 'UNI3665884049186',
 'UNI3665884049193',
 'UNI3665884049209',
 'UNI3665884049216',
 'UNI3665884049223',
 'UNI3665884049445',
 'UNI3665884049452',
 'UNI3665884049469',
 'UNI3665884049476',
 'UNI3665884049483',
 'UNI3665884049490',
 'UNI3665884049506',
 'UNI3665884049520',
 'UNI3665884049537',
 'UNI3665884049544',
 'UNI3665884049568',
 'UNI3665884049575',
 'UNI3665884049582',
 'UNI3665884052360',
 'UNI3665884063168',
 'UNI3665884063212',
 'UNI3665884063229',
 'UNI3665884063236',
 'UNI3665884063250',
 'UNI3665884063281',
 'UNI3665884063304',
 'UNI3665884063328',
 'UNI3665884063359',
 'UNI3665884063427',
 'UNI3665884063434',
 'UNI3665884063472',
 'UNI3665884063489',
 'UNI3665884067098',
 'UNI3665884067180',
 'UNI3665884067197',
 'UNI3665884067241',
 'UNI3665884067258',
 'UNI3665884067265',
 'UNI3665884067289',
 'UNI3665884067296',
 'UNI3665884067319',
 'UNI3665884067326',
 'UNI3665884067333',
 'UNI3665884067357',
 'UNI3665884067364',
 'UNI36658840

In [96]:
item_list_10_series = pd.Series(item_list_10)
item_list_10_series.value_counts().sum()

np.int64(461)

In [35]:
# Crée les options pour Chrome
chrome_options = Options()
# Ajoutez les options souhaitées
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Initialise le WebDriver avec les options
driver = webdriver.Chrome(options=chrome_options)

#Initialize an empty list to store the URLs
url_list = []

for item in item_list:
    # Construct the initial URL for each product
    initial_url = f"https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{item}.html"
    
    # Load the webpage
    driver.get(initial_url)
    
    # Retrieve the final URL after the page has fully loaded
    final_url = driver.current_url
    
    # Print the final URL
    print(f"The final URL is: {final_url}")

    # Add the final URL to the list
    url_list.append(final_url)  # Correction: place this inside the loop

#Close the WebDriver
driver.quit()

The final URL is: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-003411COBURG.html
The final URL is: https://www.cdiscount.com/maison/meubles-mobilier/essencial-meuble-tv-avec-led-classique-blanc-brill/f-11760010502-016642f.html
The final URL is: https://www.cdiscount.com/maison/meubles-mobilier/buffet-bas-panneaux-de-particules-decor-chene/f-117600902-0191enfi.html
The final URL is: https://www.cdiscount.com/maison/meubles-mobilier/buffet-enfilade-cesar-decor-chene-beige-grise-et/f-117600902-0355e4pt.html
The final URL is: https://www.cdiscount.com/maison/meubles-mobilier/ensemble-enfilade-table-loft-melamine-decor-ch/f-117600904-0397ense.html


In [97]:
# Configurer le logging
logging.basicConfig(
    filename='script_log.log',  # Nom du fichier de log
    level=logging.INFO,        # Niveau de logging
    format='%(asctime)s - %(levelname)s - %(message)s'  # Format des messages de log
)

# Charger les user agents depuis le fichier YAML
try:
    with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/user_agents.yml', 'r') as f:
        user_agents = yaml.safe_load(f)
except Exception as e:
    logging.error(f"Erreur lors du chargement des user agents : {e}")
    raise

def get_random_user_agent():
    try:
        browser = random.choice(list(user_agents.keys()))
        return random.choice(user_agents[browser])
    except Exception as e:
        logging.error(f"Erreur lors de la sélection du user agent : {e}")
        raise

# Configurer les options Chrome
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Décommentez si vous voulez exécuter Chrome en mode sans tête
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument(f"user-agent={get_random_user_agent()}")

# Initialize the WebDriver (e.g., for Chrome)
try:
    driver = webdriver.Chrome(options=chrome_options)
except Exception as e:
    logging.error(f"Erreur lors de l'initialisation de WebDriver : {e}")
    raise

# Initialize an empty list to store the URLs
url_list = []

url_formats = [
    "https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{item}.html",
    "https://www.cdiscount.com/bricolage/outillage/black-decker-perceuse-visseuse-sans-fil-18-v-li-io/f-166010101-{item}.html",
    "https://www.cdiscount.com/electromenager/beaute-masculine/tondeuse-philips-oneblade-pro-360-visage-et-corp/f-1105805-{item}.html",
    "https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-{item}.html",
    "https://www.cdiscount.com/telephonie/telephone-mobile/samsung-galaxy-a15-smartphone-128go-bleu-nuit/f-1440402-{item}.html",
    "https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-{item}.html",
    "https://www.cdiscount.com/high-tech/casques-baladeur-hifi/cowin-e7-casque-audio-bluetooth-sans-fil-arceau/f-1065420-{item}.html",
    "https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-{item}.html",
    "https://www.cdiscount.com/le-sport/mobilite-urbaine/kukirin-g2-master-trottinette-electrique-pliable/f-1214801-{item}.html",
    "https://www.cdiscount.com/maison/canape-canapes/canape-d-angle-droit-convertible-coffre-5-places/f-11701-{item}.html",
    "https://www.cdiscount.com/le-sport/velos-tandem-tricycle-monocycle-remorque-casque/tacx-home-trainer-pour-velo-tacx-boost/f-121192411-{item}.html",
    "https://www.cdiscount.com/pret-a-porter/bebe-puericulture/moonmini-babyphone-120deg-moniteur-3-2-lcd-co/f-113173701-{item}.html",
    "https://www.cdiscount.com/jardin/mobilier-de-jardin/ensemble-repas-de-jardin-pour-enfant-2-personnes/f-1631601-{item}.html",
    "https://www.cdiscount.com/jardin/mobilier-de-jardin/chilienne-de-jardin-en-bois-avec-assise-en-polyest/f-1631609-{item}.html",
    "https://www.cdiscount.com/maison/achat-meuble-literie/sommier-tapissier-140x190cm-frixovel-sommiers-a/f-1175527-{item}.html",
    "https://www.cdiscount.com/maison/lampes/suspension-naturelle-en-bambou-beige-40-x-38-c/f-1170206-{item}.html",
    "https://www.cdiscount.com/maison/meubles-mobilier/commode-chelsea-3-tiroirs-couleur-chene-l-77-2/f-117600211-{item}.html",
    "https://www.cdiscount.com/maison/fauteuil-pouf-poire/fauteuil-de-bureau-repose-pieds-telescopique-mec/f-117200101-{item}.html",
    "https://www.cdiscount.com/maison/meubles-mobilier/vasagle-bibliotheque-avec-14-compartiments-style/f-117600704-{item}.html",
    "https://www.cdiscount.com/maison/meubles-mobilier/etagere-bibliotheque-detroit-design-industriel/f-117600704-{item}.html",
    "https://www.cdiscount.com/maison/linge-maison/lot-de-2-antiderapant-ancien-tapis-de-cuisine-tapi/f-117620206-{item}.html"
]

# Nombre total d'éléments à traiter
total_items = len(item_list_10)
completed_items = 0

for item in item_list_10:
    try:
        # Choisir un format d'URL aléatoire
        url_format = random.choice(url_formats)
        
        # Construct the initial URL for each product
        initial_url = url_format.format(item=item)

        # Load the webpage
        driver.get(initial_url)
        
        # Retrieve the final URL after the page has fully loaded
        final_url = driver.current_url
        
        # Print the final URL
        print(f"The final URL is: {final_url}")

        # Add the final URL to the list
        url_list.append(final_url)  # Correction: place this inside the loop
        
        # Mise à jour du suivi de la progression
        completed_items += 1
        percentage = (completed_items / total_items) * 100
        print(f"[{completed_items}/{total_items}] ({percentage:.2f}%) Final URL for product {item}: {final_url}")
    
    except Exception as e:
        logging.error(f"Erreur lors du traitement de l'URL pour l'article {item} : {e}")

# Close the WebDriver
driver.quit()


The final URL is: https://www.cdiscount.com/maison/decoration-accessoires/voilage-panama-135-x-250-cm-taupe-1-panneau/f-117630805-uni3665884035592.html
[1/461] (0.22%) Final URL for product UNI3665884035592: https://www.cdiscount.com/maison/decoration-accessoires/voilage-panama-135-x-250-cm-taupe-1-panneau/f-117630805-uni3665884035592.html
The final URL is: https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-indi-140-x-240/f-117630805-uni3665884049179.html
[2/461] (0.43%) Final URL for product UNI3665884049179: https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-indi-140-x-240/f-117630805-uni3665884049179.html
The final URL is: https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-indi-140-x-240/f-117630805-uni3665884049186.html
[3/461] (0.65%) Final URL for product UNI3665884049186: https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-ind

In [98]:
url_list

['https://www.cdiscount.com/maison/decoration-accessoires/voilage-panama-135-x-250-cm-taupe-1-panneau/f-117630805-uni3665884035592.html',
 'https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-indi-140-x-240/f-117630805-uni3665884049179.html',
 'https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-indi-140-x-240/f-117630805-uni3665884049186.html',
 'https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-lignes-horizontales-indi-140-x-240/f-117630805-uni3665884049193.html',
 'https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-sable-alton-140-x-240-cm-atmospher/f-117630805-uni3665884049209.html',
 'https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-sable-alton-140-x-240-cm-atmospher/f-117630805-uni3665884049216.html',
 'https://www.cdiscount.com/maison/decoration-accessoires/voilage-tisse-sable-alton-140-x-240-cm-atmospher/f-117630805-uni3665884049223.html',
 'ht

In [99]:
# Create a temporary DataFrame with the results
# #df_temp = pd.DataFrame({
#     'product_id': item_list,
#     'url': url_list
# })

#df_temp[['domain', 'category', 'subcategory', 'product_name']] = df_temp['url'].str.split('/', expand=True).iloc[:, 2:6]
url_series = pd.Series(url_list)
display(url_series)
product_list_10 = pd.DataFrame({
    'product_id': url_series.str.split('/', expand=True).iloc[:,-1].str.split('.', expand=True).iloc[:,0].str.split('-', expand=True).iloc[:,-1],
    'url': url_series
})

product_list_10.to_csv('product_list_10.csv', index=False)

# Display the temporary DataFrame to verify
product_list_10.head()

0      https://www.cdiscount.com/maison/decoration-ac...
1      https://www.cdiscount.com/maison/decoration-ac...
2      https://www.cdiscount.com/maison/decoration-ac...
3      https://www.cdiscount.com/maison/decoration-ac...
4      https://www.cdiscount.com/maison/decoration-ac...
                             ...                        
456    https://www.cdiscount.com/bricolage/electricit...
457    https://www.cdiscount.com/bricolage/electricit...
458    https://www.cdiscount.com/electromenager/soin-...
459    https://www.cdiscount.com/electromenager/petit...
460    https://www.cdiscount.com/bricolage/securite-d...
Length: 461, dtype: object

,product_id,url
0,uni3665884035592,https://www.cdiscount.com/maison/decoration-ac...
1,uni3665884049179,https://www.cdiscount.com/maison/decoration-ac...
2,uni3665884049186,https://www.cdiscount.com/maison/decoration-ac...
3,uni3665884049193,https://www.cdiscount.com/maison/decoration-ac...
4,uni3665884049209,https://www.cdiscount.com/maison/decoration-ac...


In [2]:
import os

# Initialiser une liste pour stocker tous les DataFrames
df_list = []

# Lire chaque fichier CSV et extraire les colonnes souhaitées
for i in range(11):
    # Créer le nom du fichier
    file_name = f'product_list_{i}.csv' if i > 0 else 'product_list.csv'
    
    # Lire le fichier CSV
    df_temp = pd.read_csv(file_name)
    
    # Séparer les parties de l'URL en colonnes distinctes
    df_temp[['domain', 'category', 'subcategory', 'product_name']] = df_temp['url'].str.split('/', expand=True).iloc[:, 2:6]
    
    # Ajouter le DataFrame à la liste
    df_list.append(df_temp)

# Concaténer tous les DataFrames en un seul
df_combined = pd.concat(df_list, ignore_index=True)

# Afficher le DataFrame combiné
df_combined.head()

NameError: name 'pd' is not defined

In [47]:
# Séparer les parties de l'URL en colonnes distinctes
df_temp[['domain', 'category', 'subcategory', 'product_name']] = df_temp['url'].str.split('/', expand=True).iloc[:, 2:6]

# Afficher le DataFrame mis à jour
df_temp.head()

,product_id,url,domain,category,subcategory,product_name
0,003411COBURG,https://www.cdiscount.com/maison/meubles-mobil...,www.cdiscount.com,maison,meubles-mobilier,table-extensible-coburg-1-allonge-40cm-style-i
1,016642F,https://www.cdiscount.com/maison/meubles-mobil...,www.cdiscount.com,maison,meubles-mobilier,essencial-meuble-tv-avec-led-classique-blanc-b...
2,0191ENFI,https://www.cdiscount.com/maison/meubles-mobil...,www.cdiscount.com,maison,meubles-mobilier,buffet-bas-panneaux-de-particules-decor-chene
3,0355E4PT,https://www.cdiscount.com/maison/meubles-mobil...,www.cdiscount.com,maison,meubles-mobilier,buffet-enfilade-cesar-decor-chene-beige-grise-et
4,0397ENSE,https://www.cdiscount.com/maison/meubles-mobil...,www.cdiscount.com,maison,meubles-mobilier,ensemble-enfilade-table-loft-melamine-decor-ch


## Scraping anti détection


In [30]:
# Charger les fichiers YAML
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/headers.yml', 'r') as f:
    headers = yaml.safe_load(f)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/user_agents.yml', 'r') as f:
    user_agents = yaml.safe_load(f)

# Afficher les données pour vérification
print(headers)
print(user_agents)

{'Chrome': {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'accept-encoding': 'gzip, deflate, br', 'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7', 'cache-control': 'max-age=0', 'sec-fetch-dest': 'document', 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'same-origin', 'sec-fetch-user': '?1', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}, 'Edge': {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'accept-encoding': 'gzip, deflate, br', 'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8', 'cache-control': 'max-age=0', 'sec-fetch-dest': 'document', 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'none', 'sec-fetch-user': '?1', 'upgrade-insecure-requests': '1', 'user-agent': 'Mo

In [14]:
def get_random_user_agent():
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

In [15]:
def fetch_proxies():
    # Récupérer la liste des proxies
    response = requests.get("https://free-proxy-list.net/")
    proxy_list = pd.read_html(response.text)[0]
    proxy_list["url"] = "http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
    https_proxies = proxy_list[proxy_list["Https"] == "yes"]

    # Tester les proxies
    url = "https://httpbin.org/ip"
    good_proxies = set()
    headers_chrome = headers["Chrome"]

    for proxy_url in https_proxies["url"]:
        proxies = {
            "http": proxy_url,
            "https": proxy_url,
        }
        try:
            response = requests.get(url, headers=headers_chrome, proxies=proxies, timeout=2)
            good_proxies.add(proxy_url)
            print(f"Proxy {proxy_url} OK, added to good_proxy list")
        except Exception:
            pass
        if len(good_proxies) >= 10:  # Ajuster selon les besoins
            break

    return list(good_proxies)

# Récupérer et afficher les proxies valides
proxies = fetch_proxies()
print(proxies)

/var/folders/hm/xzr5dzsn36x5q3_9gvhtjqm00000gn/T/ipykernel_19266/3641518889.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  proxy_list = pd.read_html(response.text)[0]


Proxy http://135.148.100.78:48149 OK, added to good_proxy list
['http://135.148.100.78:48149']


In [35]:
def get_random_proxy():
    return random.choice(proxies)

In [34]:
def fetch_final_url(product_id, proxy, user_agent):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    
    # Configurer le proxy
    chrome_options.add_argument(f"--proxy-server={proxy}")
    
    # Configurer le user-agent
    chrome_options.add_argument(f"user-agent={user_agent}")
    
    # Chemin vers le ChromeDriver (remplacez avec le chemin approprié)
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        initial_url = f"https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html"
        driver.get(initial_url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        final_url = driver.current_url
        print(f"Final URL for product {product_id}: {final_url}")
        return final_url
    except Exception as e:
        print(f"Erreur pour l'ID {product_id}: {e}")
        return None
    finally:
        driver.quit()

# Tester avec un exemple de product_id, proxy et user_agent
test_product_id = df_pdt_id['product_id'].iloc[9]
test_proxy = get_random_proxy()  # Assurez-vous que cette fonction existe
test_user_agent = get_random_user_agent()  # Assurez-vous que cette fonction existe
fetch_final_url(test_product_id, test_proxy, test_user_agent)

Final URL for product 03K5386486: https://www.cdiscount.com/maison/meubles-mobilier/buffet-haut-decor-graphite-et-chene-3-portes/f-117600902-03k5386486.html


'https://www.cdiscount.com/maison/meubles-mobilier/buffet-haut-decor-graphite-et-chene-3-portes/f-117600902-03k5386486.html'

In [ ]:
def parallel_scraping(item_list, max_workers=10):
    url_list = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_id = {executor.submit(fetch_final_url, item, get_random_proxy(), get_random_user_agent()): item for item in item_list}
        
        for future in as_completed(future_to_id):
            product_id = future_to_id[future]
            try:
                final_url = future.result()
                if final_url:
                    url_list.append(final_url)
                    print(f"Final URL for product {product_id}: {final_url}")
            except Exception as exc:
                print(f"Erreur pour l'ID {product_id}: {exc}")
            
            # Attendre un temps aléatoire entre les requêtes pour éviter la détection
            time.sleep(random.uniform(1, 3))
    
    return url_list

# Exécution de la parallélisation
url_list = parallel_scraping(df_pdt_id['product_id'].tolist(), max_workers=10)
print(url_list)

# Sauvegarder les URLs finales dans un fichier CSV
url_df = pd.DataFrame(url_list, columns=['Final_URL'])

Final URL for product 003411COBURG: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-003411COBURG.html
Final URL for product 003411COBURG: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-003411COBURG.html
Erreur pour l'ID 0355E4PT: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: chrome-headless-shell=127.0.6533.120)
Stacktrace:
0   chromedriver                        0x000000010525d024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000105255700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x0000000104e6482c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000104e5d7e8 cxxbridge1$string$len + 59784
4   chromedriver                        0x0000000104e50008 cxxbridge1$string$len + 4520
5   chromedriver                        0x0000000104e51a10 cxxbridge1$string$len + 1118


KeyboardInterrupt



Final URL for product BEK8690842609770: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-BEK8690842609770.html
Erreur pour l'ID BERGEN002850: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: chrome-headless-shell=127.0.6533.120)
Stacktrace:
0   chromedriver                        0x0000000104a21024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000104a19700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010462882c cxxbridge1$string$len + 88524
3   chromedriver                        0x00000001046217e8 cxxbridge1$string$len + 59784
4   chromedriver                        0x0000000104614008 cxxbridge1$string$len + 4520
5   chromedriver                        0x0000000104615a10 cxxbridge1$string$len + 11184
6   chromedriver                        0x00000001046143e4 cxxbridge1$string$len + 5508
7   chromedriver                        0x0000000104613c30 cx

In [36]:
# Convertir la liste des URLs en DataFrame
url_df = pd.DataFrame(url_list)

# Joindre les données des URLs au DataFrame d'origine en utilisant 'product_id'
df_pdt_id = df_pdt_id.merge(url_df, on='product_id', how='left')

# Vérifier que les URLs ont bien été ajoutées
print(df_pdt_id.head())

# Séparer l'URL après 'cdiscount.com/' en colonnes
# Assurez-vous que la colonne 'url' ne contient pas de valeurs nulles
df_pdt_id['url'] = df_pdt_id['url'].fillna('')
df_pdt_id[['domain', 'category', 'subcategory', 'product_name']] = df_pdt_id['url'].str.split('/', expand=True).iloc[:, 2:6]

# Afficher le DataFrame mis à jour
print(df_pdt_id.head())

NameError: name 'url_list' is not defined

In [ ]:
# Sauvegarder les URLs finales dans un fichier CSV
url_df = pd.DataFrame(url_list, columns=['Final_URL'])
url_df.to_csv('final_urls.csv', index=False)

In [ ]:
# Charger les fichiers YAML
with open('headers.yml', 'r') as f:
    headers = yaml.safe_load(f)

with open('user-agents.yaml', 'r') as f:
    user_agents = yaml.safe_load(f)

def get_random_user_agent():
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

def fetch_proxies():
    response = requests.get("https://free-proxy-list.net/")
    proxy_list = pd.read_html(response.text)[0]
    proxy_list["url"] = "http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
    https_proxies = proxy_list[proxy_list["Https"] == "yes"]

    url = "https://httpbin.org/ip"
    good_proxies = set()
    headers_chrome = headers["Chrome"]
    
    for proxy_url in https_proxies["url"]:
        proxies = {
            "http": proxy_url,
            "https": proxy_url,
        }
        try:
            response = requests.get(url, headers=headers_chrome, proxies=proxies, timeout=2)
            good_proxies.add(proxy_url)
            print(f"Proxy {proxy_url} OK, added to good_proxy list")
        except Exception:
            pass
        if len(good_proxies) >= 10:  # Ajuste selon tes besoins
            break
    
    return list(good_proxies)

proxies = fetch_proxies()

def get_random_proxy():
    return random.choice(proxies)

def fetch_final_url(product_id):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    
    # Configurer le proxy
    proxy = get_random_proxy()
    chrome_options.add_argument(f"--proxy-server={proxy}")
    
    # Configurer le user-agent
    user_agent = get_random_user_agent()
    chrome_options.add_argument(f"user-agent={user_agent}")
    
    # Chemin vers le ChromeDriver (remplacez avec le chemin approprié)
    service = Service('/path/to/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        initial_url = f"https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html"
        driver.get(initial_url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        final_url = driver.current_url
        return final_url
    except Exception as e:
        print(f"Erreur pour l'ID {product_id}: {e}")
        return None
    finally:
        driver.quit()

def parallel_scraping(item_list, max_workers=10):
    url_list = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_id = {executor.submit(fetch_final_url, item): item for item in item_list}
        
        for future in as_completed(future_to_id):
            product_id = future_to_id[future]
            try:
                final_url = future.result()
                if final_url:
                    url_list.append(final_url)
                    print(f"Final URL for product {product_id}: {final_url}")
            except Exception as exc:
                print(f"Erreur pour l'ID {product_id}: {exc}")
            
            # Attendre un temps aléatoire entre les requêtes pour éviter la détection
            time.sleep(random.uniform(1, 3))
    
    return url_list

# Exécution de la parallélisation
url_list = parallel_scraping(item_list, max_workers=10)

# Sauvegarder les URLs finales dans un fichier CSV
url_df = pd.DataFrame(url_list, columns=['Final_URL'])
url_df.to_csv('final_urls.csv', index=False)

In [57]:
# Charger les fichiers YAML
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/headers.yml', 'r') as f:
    headers = yaml.safe_load(f)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/user_agents.yml', 'r') as f:
    user_agents = yaml.safe_load(f)

def get_random_user_agent():
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

def fetch_proxies():
    response = requests.get("https://free-proxy-list.net/")
    proxy_list = pd.read_html(response.text)[0]
    proxy_list["url"] = "http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
    https_proxies = proxy_list[proxy_list["Https"] == "yes"]

    url = "https://httpbin.org/ip"
    good_proxies = set()
    headers_chrome = headers["Chrome"]
    
    for proxy_url in https_proxies["url"]:
        proxies = {
            "http": proxy_url,
            "https": proxy_url,
        }
        try:
            response = requests.get(url, headers=headers_chrome, proxies=proxies, timeout=2)
            good_proxies.add(proxy_url)
            print(f"Proxy {proxy_url} OK, added to good_proxy list")
        except Exception:
            pass
        if len(good_proxies) >= 10:  # Ajuste selon tes besoins
            break
    
    return list(good_proxies)

proxies = fetch_proxies()

def get_random_proxy():
    return random.choice(proxies)

def fetch_final_url(product_id):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")

    # Liste de base URLs
    base_urls = [
        "https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html",
        "https://www.cdiscount.com/bricolage/outillage/black-decker-perceuse-visseuse-sans-fil-18-v-li-io/f-166010101-{product_id}.html",
        "https://www.cdiscount.com/electromenager/beaute-masculine/tondeuse-philips-oneblade-pro-360-visage-et-corp/f-1105805-{product_id}.html",
        "https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-{product_id}.html",
        "https://www.cdiscount.com/telephonie/telephone-mobile/samsung-galaxy-a15-smartphone-128go-bleu-nuit/f-1440402-{product_id}.html",
        "https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-{product_id}.html",
        "https://www.cdiscount.com/high-tech/casques-baladeur-hifi/cowin-e7-casque-audio-bluetooth-sans-fil-arceau/f-1065420-{product_id}.html",
        "https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-{product_id}.html",
        "https://www.cdiscount.com/le-sport/mobilite-urbaine/kukirin-g2-master-trottinette-electrique-pliable/f-1214801-{product_id}.html",
        "https://www.cdiscount.com/maison/canape-canapes/canape-d-angle-droit-convertible-coffre-5-places/f-11701-{product_id}.html",
        "https://www.cdiscount.com/le-sport/velos-tandem-tricycle-monocycle-remorque-casque/tacx-home-trainer-pour-velo-tacx-boost/f-121192411-{product_id}.html",
        "https://www.cdiscount.com/pret-a-porter/bebe-puericulture/moonmini-babyphone-120deg-moniteur-3-2-lcd-co/f-113173701-{product_id}.html"
    ]
    
    # Choisir une URL de base aléatoire et remplacer le {product_id}
    base_url = random.choice(base_urls)
    url = base_url.format(product_id=product_id)
    
    # Configurer le proxy
    proxy = get_random_proxy()
    chrome_options.add_argument(f"--proxy-server={proxy}")
    
    # Configurer le user-agent
    user_agent = get_random_user_agent()
    chrome_options.add_argument(f"user-agent={user_agent}")

    # Chemin vers le ChromeDriver (remplacez avec le chemin approprié)
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        driver.get(url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        final_url = driver.current_url
        return final_url
    except Exception as e:
        print(f"Erreur pour l'URL {url}: {e}")
        return None
    finally:
        driver.quit()

def parallel_scraping(item_list, max_workers=10):
    final_urls = []
    total_items = len(item_list)
    completed_items = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_item = {executor.submit(fetch_final_url, item): item for item in item_list}
        
        for future in as_completed(future_to_item):
            product_id = future_to_item[future]
            try:
                final_url = future.result()
                if final_url:
                    final_urls.append(final_url)
                    completed_items += 1
                    percentage = (completed_items / total_items) * 100
                    print(f"[{completed_items}/{total_items}] ({percentage:.2f}%) Final URL for product {product_id}: {final_url}")
            except Exception as exc:
                print(f"Erreur pour l'ID {product_id}: {exc}")
            
            # Attendre un temps aléatoire entre les requêtes pour éviter la détection
            time.sleep(random.uniform(1, 3))
    
    return final_urls

# Utiliser le DataFrame existant pour obtenir les IDs de produit
item_list = df_pdt_id['product_id'].tolist()

# Exécution de la parallélisation
final_urls = parallel_scraping(item_list, max_workers=10)

# Manipuler les URLs finales avec un DataFrame
df_pdt_id['url'] = final_urls

/var/folders/hm/xzr5dzsn36x5q3_9gvhtjqm00000gn/T/ipykernel_12164/3453375447.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  proxy_list = pd.read_html(response.text)[0]


Erreur pour l'ID DEM3397701900072: Cannot choose from an empty sequence
Erreur pour l'ID POL8007411010835: Cannot choose from an empty sequence
Erreur pour l'ID VARIERELAYON: Cannot choose from an empty sequence
Erreur pour l'ID KAR4054278874579: Cannot choose from an empty sequence
Erreur pour l'ID AUC3701298844822: Cannot choose from an empty sequence
Erreur pour l'ID IRI8716382198670: Cannot choose from an empty sequence
Erreur pour l'ID GRE8435574354990: Cannot choose from an empty sequence
Erreur pour l'ID AUC3665884031754: Cannot choose from an empty sequence
Erreur pour l'ID EIN4006825637496: Cannot choose from an empty sequence
Erreur pour l'ID GRO4005176521171: Cannot choose from an empty sequence
Erreur pour l'ID BAI3584179052963: Cannot choose from an empty sequence
Erreur pour l'ID AME3546690512517: Cannot choose from an empty sequence
Erreur pour l'ID DAV3664632022570: Cannot choose from an empty sequence
Erreur pour l'ID DAV2008458541241: Cannot choose from an empty seque

KeyboardInterrupt: 

In [ ]:
# Sample DataFrame for demonstration
# df_pdt_id = pd.DataFrame({'product_id': [1, 2, 3], 'url': url_list})

# Assuming df_pdt_id already exists and contains 'url' column
df_pdt_id['url'] = url_list

# Split the URL after 'cdiscount.com/' into separate columns
df_pdt_id[['domain', 'category', 'subcategory', 'product_name']] = df_pdt_id['url'].str.split('/', expand=True).iloc[:, 2:6]

# Display the updated DataFrame
df_pdt_id.head()

In [ ]:
# Séparer les parties de l'URL en colonnes distinctes
df_pdt_id[['domain', 'category', 'subcategory', 'product_name']] = df_pdt_id['url'].str.split('/', expand=True).iloc[:, 2:6]

# Afficher le DataFrame mis à jour
print(df_pdt_id.head())

# Correction du scraping
Problème de Proxy - Modification et utilisation de Tor avec Selenium

In [2]:
#!pip install stem fake-useragent

In [5]:
from stem import Signal
from stem.control import Controller
from fake_useragent import UserAgent
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from time import sleep

In [4]:
def renew_ip():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)
    sleep(7)  # Attendre que le changement d'IP prenne effet

# Exemple d'utilisation
renew_ip()

In [20]:
def profil_proxy():
    ip_tor = "127.0.0.1"  # Localhost pour la machine locale
    port_tor = 9050
    proxy = f"socks5://{ip_tor}:{port_tor}"
    return proxy

def renew_ip():
    with Controller.from_port(address='127.0.0.1', port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)
        print("Signal NEWNYM envoyé.")
    time.sleep(10)  # Augmenter le délai pour donner plus de temps pour le renouvellement


def create_driver():
    proxy = profil_proxy()
    
    options = Options()
    options.headless = True
    options.add_argument(f'--proxy-server={proxy}')
    options.add_argument(f'--proxy-server=socks5://127.0.0.1:9050')

    service = Service('/usr/local/bin/chromedriver')  # Remplacez par le chemin de votre chromedriver
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_final_url_with_tor(url):
    driver = create_driver()
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        final_url = driver.current_url
        return final_url
    except Exception as e:
        print(f"Erreur pour l'URL {url}: {e}")
        return None
    finally:
        driver.quit()

def fetch_url_with_requests(url):
    proxies = {'http': 'socks5://127.0.0.1:9050', 'https': 'socks5://127.0.0.1:9050'}
    headers = {'User-Agent': UserAgent().random}
    response = requests.get(url, proxies=proxies, headers=headers)
    print(response.text)
    print(response.request.headers)
    return response.text

# # Exemple d'utilisation
# url = 'https://api.ipify.org'
# response_text = fetch_url_with_requests(url)
# print(f"Réponse: {response_text}")

url = 'https://httpbin.org/ip'
response_text = fetch_url_with_requests(url)
print(f"Réponse: {response_text}")

# Renouveler l'adresse IP
renew_ip()

# Obtenir la nouvelle URL via Tor
final_url = fetch_final_url_with_tor(url)
print(f"Final URL: {final_url}")

{
  "origin": "107.189.8.133"
}

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
Réponse: {
  "origin": "107.189.8.133"
}

Signal NEWNYM envoyé.
Final URL: https://httpbin.org/ip


---
## GOOD CODE CI-DESSOUS
---

In [12]:
# Charger les fichiers YAML
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/headers.yml', 'r') as f:
    headers = yaml.safe_load(f)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/user_agents.yml', 'r') as f:
    user_agents = yaml.safe_load(f)

def get_random_user_agent():
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

def renew_ip():
    with Controller.from_port(address='127.0.0.1', port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)
    time.sleep(7)  # Attendre que le nouvel IP soit actif

def create_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--proxy-server=socks5://127.0.0.1:9050")
    
    # Configurer le user-agent
    user_agent = get_random_user_agent()
    chrome_options.add_argument(f"user-agent={user_agent}")

    # Chemin vers le ChromeDriver (remplacez avec le chemin approprié)
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def fetch_final_url(product_id):
    # Liste de base URLs
    base_urls = [
        "https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html",
        "https://www.cdiscount.com/bricolage/outillage/black-decker-perceuse-visseuse-sans-fil-18-v-li-io/f-166010101-{product_id}.html",
        "https://www.cdiscount.com/electromenager/beaute-masculine/tondeuse-philips-oneblade-pro-360-visage-et-corp/f-1105805-{product_id}.html",
        "https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-{product_id}.html",
        "https://www.cdiscount.com/telephonie/telephone-mobile/samsung-galaxy-a15-smartphone-128go-bleu-nuit/f-1440402-{product_id}.html",
        "https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-{product_id}.html",
        "https://www.cdiscount.com/high-tech/casques-baladeur-hifi/cowin-e7-casque-audio-bluetooth-sans-fil-arceau/f-1065420-{product_id}.html",
        "https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-{product_id}.html",
        "https://www.cdiscount.com/le-sport/mobilite-urbaine/kukirin-g2-master-trottinette-electrique-pliable/f-1214801-{product_id}.html",
        "https://www.cdiscount.com/maison/canape-canapes/canape-d-angle-droit-convertible-coffre-5-places/f-11701-{product_id}.html",
        "https://www.cdiscount.com/le-sport/velos-tandem-tricycle-monocycle-remorque-casque/tacx-home-trainer-pour-velo-tacx-boost/f-121192411-{product_id}.html",
        "https://www.cdiscount.com/pret-a-porter/bebe-puericulture/moonmini-babyphone-120deg-moniteur-3-2-lcd-co/f-113173701-{product_id}.html"
    ]
    
    # Choisir une URL de base aléatoire et remplacer le {product_id}
    base_url = random.choice(base_urls)
    url = base_url.format(product_id=product_id)
    
    driver = create_driver()
    
    try:
        driver.get(url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Attendre que l'URL actuelle soit modifiée ou que certains éléments clés soient présents
        WebDriverWait(driver, 20).until(
            lambda d: d.current_url != url  # Attendre que l'URL change ou que certains éléments clés apparaissent
        )
        
        final_url = driver.current_url
        return final_url
    except Exception as e:
        print(f"Erreur pour l'URL {url}: {e}")
        return None
    finally:
        driver.quit()

def parallel_scraping(item_list, max_workers=10):
    final_urls = []
    total_items = len(item_list)
    completed_items = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_item = {executor.submit(fetch_final_url, item): item for item in item_list}
        
        for future in as_completed(future_to_item):
            product_id = future_to_item[future]
            try:
                final_url = future.result()
                if final_url:
                    final_urls.append(final_url)
                    completed_items += 1
                    percentage = (completed_items / total_items) * 100
                    print(f"[{completed_items}/{total_items}] ({percentage:.2f}%) Final URL for product {product_id}: {final_url}")
            except Exception as exc:
                print(f"Erreur pour l'ID {product_id}: {exc}")
            
            # Attendre un temps aléatoire entre les requêtes pour éviter la détection
            time.sleep(random.uniform(1, 3))
    
    return final_urls

# Utiliser le DataFrame existant pour obtenir les IDs de produit
# item_list = df_pdt_id['product_id'].tolist()
# Utiliser le DataFrame existant pour obtenir un échantillon aléatoire de 20 IDs de produit
item_list = df_pdt_id['product_id'].sample(n=5, random_state=1).tolist()

# Exécution de la parallélisation
final_urls = parallel_scraping(item_list, max_workers=10)

Erreur pour l'URL https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-VAS6955880328512.html: Message: 

Erreur pour l'URL https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-AUC3701071155305.html: Message: 

Erreur pour l'URL https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-CTR0782846975161.html: Message: 

Erreur pour l'URL https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-UNI3665884067265.html: Message: 

Erreur pour l'URL https://www.cdiscount.com/pret-a-porter/bebe-puericulture/moonmini-babyphone-120deg-moniteur-3-2-lcd-co/f-113173701-DAV3664632020637.html: Message: 



In [8]:
# Créer un DataFrame temporaire avec les résultats
df_temp = pd.DataFrame({
    'product_id': item_list,
    'url': final_urls
})

# Afficher le DataFrame temporaire pour vérifier
df_temp.head()

NameError: name 'final_urls' is not defined

In [28]:
# Séparer les parties de l'URL en colonnes distinctes
df_temp[['domain', 'category', 'subcategory', 'product_name']] = df_temp['url'].str.split('/', expand=True).iloc[:, 2:6]

# Afficher le DataFrame mis à jour
df_temp.head()

,product_id,url,domain,category,subcategory,product_name
0,UNI3665884067265,https://www.cdiscount.com/high-tech/casques-ba...,www.cdiscount.com,high-tech,casques-baladeur-hifi,cowin-e7-casque-audio-bluetooth-sans-fil-arceau
1,AUC3701071155305,https://www.cdiscount.com/telephonie/telephone...,www.cdiscount.com,telephonie,telephone-mobile,samsung-galaxy-a15-smartphone-128go-bleu-nuit
2,CTR0782846975161,https://www.cdiscount.com/maison/canape-canape...,www.cdiscount.com,maison,canape-canapes,canape-d-angle-droit-convertible-coffre-5-places
3,VAS6955880328512,https://www.cdiscount.com/maison/meubles-mobil...,www.cdiscount.com,maison,meubles-mobilier,table-extensible-coburg-1-allonge-40cm-style-i
4,DAV3664632020637,https://www.cdiscount.com/maison/canape-canape...,www.cdiscount.com,maison,canape-canapes,canape-d-angle-droit-convertible-coffre-5-places


---

## CODE de TEST et DEBOGAGE CI-DESSOUS

---

In [13]:
# Configurer la journalisation
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Charger les fichiers YAML
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/headers.yml', 'r') as f:
    headers = yaml.safe_load(f)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/user_agents.yml', 'r') as f:
    user_agents = yaml.safe_load(f)

def get_random_user_agent():
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

def renew_ip():
    with Controller.from_port(address='127.0.0.1', port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)
    time.sleep(7)  # Attendre que le nouvel IP soit actif

def create_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--proxy-server=socks5://127.0.0.1:9050")
    
    # Configurer le user-agent
    user_agent = get_random_user_agent()
    chrome_options.add_argument(f"user-agent={user_agent}")

    # Chemin vers le ChromeDriver (remplacez avec le chemin approprié)
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def fetch_final_url(product_id):
    # Liste de base URLs
    base_urls = [
        "https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html",
        "https://www.cdiscount.com/bricolage/outillage/black-decker-perceuse-visseuse-sans-fil-18-v-li-io/f-166010101-{product_id}.html",
        "https://www.cdiscount.com/electromenager/beaute-masculine/tondeuse-philips-oneblade-pro-360-visage-et-corp/f-1105805-{product_id}.html",
        "https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-{product_id}.html",
        "https://www.cdiscount.com/telephonie/telephone-mobile/samsung-galaxy-a15-smartphone-128go-bleu-nuit/f-1440402-{product_id}.html",
        "https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-{product_id}.html",
        "https://www.cdiscount.com/high-tech/casques-baladeur-hifi/cowin-e7-casque-audio-bluetooth-sans-fil-arceau/f-1065420-{product_id}.html",
        "https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-{product_id}.html",
        "https://www.cdiscount.com/le-sport/mobilite-urbaine/kukirin-g2-master-trottinette-electrique-pliable/f-1214801-{product_id}.html",
        "https://www.cdiscount.com/maison/canape-canapes/canape-d-angle-droit-convertible-coffre-5-places/f-11701-{product_id}.html",
        "https://www.cdiscount.com/le-sport/velos-tandem-tricycle-monocycle-remorque-casque/tacx-home-trainer-pour-velo-tacx-boost/f-121192411-{product_id}.html",
        "https://www.cdiscount.com/pret-a-porter/bebe-puericulture/moonmini-babyphone-120deg-moniteur-3-2-lcd-co/f-113173701-{product_id}.html"
    ]
    
    # Choisir une URL de base aléatoire et remplacer le {product_id}
    base_url = random.choice(base_urls)
    url = base_url.format(product_id=product_id)
    
    driver = create_driver()
    
    try:
        driver.get(url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Attendre que l'URL actuelle soit modifiée ou que certains éléments clés soient présents
        WebDriverWait(driver, 20).until(
            lambda d: d.current_url != url  # Attendre que l'URL change ou que certains éléments clés apparaissent
        )
        
        final_url = driver.current_url
        logging.info(f"Final URL pour {product_id}: {final_url}")
        return final_url
    except Exception as e:
        logging.error(f"Erreur pour l'URL {url}: {e}")
        return None
    finally:
        driver.quit()
        # Attendre un temps aléatoire entre les requêtes pour éviter la détection
        time.sleep(random.uniform(5, 10))  # Attendre entre 5 et 10 secondes

def parallel_scraping(item_list, max_workers=10, requests_per_ip=5):
    final_urls = []
    total_items = len(item_list)
    completed_items = 0
    count = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_item = {executor.submit(fetch_final_url, item): item for item in item_list}
        
        for future in as_completed(future_to_item):
            product_id = future_to_item[future]
            try:
                final_url = future.result()
                if final_url:
                    final_urls.append((product_id, final_url))
                    completed_items += 1
                    percentage = (completed_items / total_items) * 100
                    logging.info(f"[{completed_items}/{total_items}] ({percentage:.2f}%) Final URL for product {product_id}: {final_url}")
            except Exception as exc:
                logging.error(f"Erreur pour l'ID {product_id}: {exc}")
            
            # Attendre un temps aléatoire entre les requêtes pour éviter la détection
            time.sleep(random.uniform(1, 3))
            
            count += 1
            if count % requests_per_ip == 0:
                renew_ip()  # Renouveler l'IP après un certain nombre de requêtes
    
    return final_urls

# Utiliser le DataFrame existant pour obtenir les IDs de produit
# item_list = df_pdt_id['product_id'].tolist()
# Utiliser le DataFrame existant pour obtenir un échantillon aléatoire de 20 IDs de produit
item_list = df_pdt_id['product_id'].sample(n=5, random_state=1).tolist()

# Exécution de la parallélisation
final_urls = parallel_scraping(item_list, max_workers=10, requests_per_ip=5)

# Convertir les résultats en DataFrame
import pandas as pd

df_results = pd.DataFrame(final_urls, columns=['product_id', 'url'])
print(df_results.head())


2024-08-23 00:44:55,222 - ERROR - Erreur pour l'URL https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-AUC3701071155305.html: Message: 

2024-08-23 00:44:55,300 - ERROR - Erreur pour l'URL https://www.cdiscount.com/le-sport/mobilite-urbaine/kukirin-g2-master-trottinette-electrique-pliable/f-1214801-DAV3664632020637.html: Message: 

2024-08-23 00:44:55,300 - ERROR - Erreur pour l'URL https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-VAS6955880328512.html: Message: 

2024-08-23 00:44:55,406 - ERROR - Erreur pour l'URL https://www.cdiscount.com/maison/canape-canapes/canape-d-angle-droit-convertible-coffre-5-places/f-11701-CTR0782846975161.html: Message: 

2024-08-23 00:44:55,800 - ERROR - Erreur pour l'URL https://www.cdiscount.com/bricolage/outillage/black-decker-perceuse-visseuse-sans-fil-18-v-li-io/f-166010101-UNI3665884067265.html: Message: 

2024-08-23 00

Empty DataFrame
Columns: [product_id, url]
Index: []


In [15]:
import requests

def test_tor_connection():
    try:
        # Test simple avec une requête HTTP via Tor
        response = requests.get('http://httpbin.org/ip', proxies={'http': 'socks5://127.0.0.1:9050', 'https': 'socks5://127.0.0.1:9050'})
        print("Réponse HTTP:", response.json())
    except requests.RequestException as e:
        print("Erreur de connexion avec Tor:", e)

# Exécuter le test
test_tor_connection()


Réponse HTTP: {'origin': '107.189.4.12'}


In [18]:
# Configuration des journaux
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def get_random_user_agent():
    # Simulez la récupération d'un user-agent aléatoire
    user_agents = {
        'chrome': ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'],
        # Ajoutez d'autres user-agents si nécessaire
    }
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

def renew_ip():
    with Controller.from_port(address='127.0.0.1', port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)
    logging.info("Renouvellement d'IP demandé.")
    time.sleep(20)  # Augmenter le délai pour s'assurer que le nouvel IP soit actif

def create_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--proxy-server=socks5://127.0.0.1:9050")
    
    # Configurer le user-agent
    user_agent = get_random_user_agent()
    chrome_options.add_argument(f"user-agent={user_agent}")

    # Chemin vers le ChromeDriver
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def fetch_final_url(product_id):
    base_urls = [
        "https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html",
    ]
    
    base_url = random.choice(base_urls)
    url = base_url.format(product_id=product_id)
    
    logging.info(f"Fetching URL: {url}")
    driver = create_driver()
    
    try:
        driver.get(url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Attendre que l'URL actuelle soit modifiée
        WebDriverWait(driver, 20).until(
            lambda d: d.current_url != url
        )
        
        final_url = driver.current_url
        logging.info(f"Final URL: {final_url}")
        return final_url
    except Exception as e:
        logging.error(f"Erreur pour l'URL {url}: {e}")
        return None
    finally:
        driver.quit()

def parallel_scraping(item_list, max_workers=10):
    final_urls = []
    total_items = len(item_list)
    completed_items = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_item = {executor.submit(fetch_final_url, item): item for item in item_list}
        
        for future in as_completed(future_to_item):
            product_id = future_to_item[future]
            try:
                final_url = future.result()
                if final_url:
                    final_urls.append(final_url)
                    completed_items += 1
                    percentage = (completed_items / total_items) * 100
                    logging.info(f"[{completed_items}/{total_items}] ({percentage:.2f}%) Final URL for product {product_id}: {final_url}")
            except Exception as exc:
                logging.error(f"Erreur pour l'ID {product_id}: {exc}")
            
            # Attendre un temps aléatoire entre les requêtes pour éviter la détection
            time.sleep(random.uniform(1, 3))
    
    return final_urls

# Utiliser le DataFrame existant pour obtenir un échantillon aléatoire de 5 IDs de produit
item_list = df_pdt_id['product_id'].sample(n=5, random_state=1).tolist()

# Exécution de la parallélisation
final_urls = parallel_scraping(item_list, max_workers=10)


2024-08-23 00:52:24,486 - INFO - Fetching URL: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-UNI3665884067265.html
2024-08-23 00:52:24,487 - INFO - Fetching URL: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-AUC3701071155305.html
2024-08-23 00:52:24,487 - INFO - Fetching URL: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-CTR0782846975161.html
2024-08-23 00:52:24,488 - INFO - Fetching URL: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-VAS6955880328512.html
2024-08-23 00:52:24,488 - INFO - Fetching URL: https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-DAV3664632020637.html
2024-08-23 00:52:38,734 - ERROR - Erreur pour l'URL https://www.cdiscount.com/maison/meubles-mobilier/table-ex

In [22]:
proxies = {
    'http': 'socks5://127.0.0.1:9050',
    'https': 'socks5://127.0.0.1:9050'
}

try:
    response = requests.get('http://httpbin.org/ip', proxies=proxies)
    print(f"IP: {response.json()['origin']}")
except requests.RequestException as e:
    print(f"Erreur de connexion : {e}")


IP: 107.189.4.12


In [ ]:
# Configuration du logging
logging.basicConfig(level=logging.INFO)

# Charger les fichiers YAML
with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/headers.yml', 'r') as f:
    headers = yaml.safe_load(f)

with open('/Users/enayargrh/Library/Mobile Documents/com~apple~CloudDocs/labo_coding/01_projects/07_cdiscount_project_da/notebooks/data/user_agents.yml', 'r') as f:
    user_agents = yaml.safe_load(f)

def get_random_user_agent():
    browser = random.choice(list(user_agents.keys()))
    return random.choice(user_agents[browser])

def create_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Décommentez pour exécuter en mode headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    
    # Configurer le user-agent
    user_agent = get_random_user_agent()
    chrome_options.add_argument(f"user-agent={user_agent}")

    # Chemin vers le ChromeDriver (remplacez avec le chemin approprié)
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def fetch_final_url(product_id):
    # Liste de base URLs
    base_urls = [
        "https://www.cdiscount.com/maison/meubles-mobilier/table-extensible-coburg-1-allonge-40cm-style-i/f-117600903-{product_id}.html",
        "https://www.cdiscount.com/bricolage/outillage/black-decker-perceuse-visseuse-sans-fil-18-v-li-io/f-166010101-{product_id}.html",
        "https://www.cdiscount.com/electromenager/beaute-masculine/tondeuse-philips-oneblade-pro-360-visage-et-corp/f-1105805-{product_id}.html",
        "https://www.cdiscount.com/maison/cuisson/tefal-ingenio-emotion-batterie-de-cuisine-10-p-in/f-1178901-{product_id}.html",
        "https://www.cdiscount.com/telephonie/telephone-mobile/samsung-galaxy-a15-smartphone-128go-bleu-nuit/f-1440402-{product_id}.html",
        "https://www.cdiscount.com/electromenager/soin-de-la-personne-beaute/5-en-1-air-styler-brosse-soufflante-brushing-set-1/f-110570503-{product_id}.html",
        "https://www.cdiscount.com/high-tech/casques-baladeur-hifi/cowin-e7-casque-audio-bluetooth-sans-fil-arceau/f-1065420-{product_id}.html",
        "https://www.cdiscount.com/vin-champagne/vin-rose/m-de-minuty-2023-cotes-de-provence-vin-rose/f-12936-{product_id}.html",
        "https://www.cdiscount.com/le-sport/mobilite-urbaine/kukirin-g2-master-trottinette-electrique-pliable/f-1214801-{product_id}.html",
        "https://www.cdiscount.com/maison/canape-canapes/canape-d-angle-droit-convertible-coffre-5-places/f-11701-{product_id}.html",
        "https://www.cdiscount.com/le-sport/velos-tandem-tricycle-monocycle-remorque-casque/tacx-home-trainer-pour-velo-tacx-boost/f-121192411-{product_id}.html",
        "https://www.cdiscount.com/pret-a-porter/bebe-puericulture/moonmini-babyphone-120deg-moniteur-3-2-lcd-co/f-113173701-{product_id}.html"
    ]
    
    # Choisir une URL de base aléatoire et remplacer le {product_id}
    base_url = random.choice(base_urls)
    url = base_url.format(product_id=product_id)
    
    driver = create_driver()
    
    try:
        driver.get(url)
        
        # Attendre que la page soit complètement chargée
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Attendre que l'URL actuelle soit modifiée ou que certains éléments clés soient présents
        WebDriverWait(driver, 10).until(
            lambda d: d.current_url != url  # Attendre que l'URL change ou que certains éléments clés apparaissent
        )
        
        final_url = driver.current_url
        return final_url
    except Exception as e:
        print(f"Erreur pour l'URL {url}: {e}")
        return None
    finally:
        driver.quit()

# Utiliser le DataFrame existant pour obtenir les IDs de produit
# item_list = df_pdt_id['product_id'].tolist()
# Utiliser le DataFrame existant pour obtenir un échantillon aléatoire de 5 IDs de produit
item_list = df_pdt_id['product_id'].sample(n=5, random_state=1).tolist()

# Extraction des URLs finales
final_urls = []

for product_id in item_list:
    final_url = fetch_final_url(product_id)
    if final_url:
        final_urls.append(final_url)
        print(f"Final URL for product {product_id}: {final_url}")
    time.sleep(random.uniform(1, 3))  # Attendre un temps aléatoire entre les requêtes
